Importa tensorflow 

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


2023-12-29 06:10:43.351028: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-29 06:10:43.351060: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-29 06:10:43.351944: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-29 06:10:43.358157: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-29 06:10:44.059985: W tensorflow/compiler/tf2

TensorFlow version: 2.15.0


Importa el resto de bibliotecas a usar

In [2]:
from Bio import SeqIO
from tensorflow import keras as kr
import numpy as np


import glob
from nombreCarpetas import ver_carpetas
import pandas as pd
import os
import psutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

2024-01-02 17:50:44.101902: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 17:50:44.101973: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 17:50:44.161601: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 17:50:44.287074: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 17:50:45.363248: W tensorflow/compiler/tf2

Funcion para buscar archivos que terminan en ".fastq", 
buscara en la direccion "path"

In [3]:
def buscar_archivos(path):
    
    fastq_files = glob.glob(f'{path}/*.fastq',recursive=True)  # This pattern assumes the files are in subdirectories, adjust it as needed

    return fastq_files

esta celda esta encargada de obtener los nombres de todos los archivos desde una carpeta

In [6]:
vc= ver_carpetas()
folders_path="../tutorialM/datos/sets3"
#folders_path="test"

nombres=vc.ver_carpetas(folders_path)
#print(f"numero carpetas:{len(nombres)}")
paths=[]
for nombre in nombres:
    paths.append(folders_path+"/"+nombre)
#print(f"ejemplo {paths[0]}")
#print(buscar_archivos_fasta(paths[0]))
#print(f"largo de paths: {len(paths)}")

Folders inside the directory
numero carpetas:301
ejemplo ../tutorialM/datos/sets3/Folder_1
['../tutorialM/datos/sets3/Folder_1/stability.trim.contigs.good.fasta']
largo de paths: 301


Funcion que devuelve el numero de veces que una "letra" (nucleotido) esta presente en una secuencia

In [7]:
def contar_letras(sequence,largo):
    possible_letters = ['A', 'C', 'G', 'T', 'N']

    letter_counts=[]
    for letter in possible_letters:
        letter_counts.append(sequence.count(letter))
    letter_counts.append(largo)
    return letter_counts

In [ ]:
def process_paired_fastq_files(file1, file2):
    sequences_r1 = [] #Lectura de fragmento de secuencia  al derecho
    sequences_r2 = [] #Lectura de fragmento de secuencia de reves
    qualities_f = []  #calidades de fragmento al derecho
    qualities_r= []   #cualidades de fragmento de reves
    ids=[]            #id del fragmento de la secuencia
    attr_fr=[]        #atributos del fragmento al derecho
    attr_rv=[]        #fragmento del fragmento de reves

    #se abren los archivos que contienen las lecturas al derecho y de reves
    with open(file1, "r") as handle1, open(file2, "r") as handle2:
        
        #se almacenan los fragmentos al derecho (record1) y de reves (record2)
        #una vez terminado, se vuelve a empezar desde el siguiente fragmento hasta que se terminen
        for record1, record2 in zip(SeqIO.parse(handle1, "fastq"), SeqIO.parse(handle2, "fastq")):
            
            #se corrobora que ambas lecturas corresponden al mismo fragmento, si no, se saltan ambos fragmentos
            assert record1.id.split()[0] == record2.id.split()[0], "Mismatch in paired reads"
            
            #Obtiene el Id de la lectura al derecho
            R1_id_mod= record1.id.split()[0].replace(":", "_")
            
            #se adjunta los fragmentos por separado
            seq1 = str(record1.seq)
            seq2 = str(record2.seq)
            
            #se almacena el largo de ambos fragmentos
            l1=len(seq1)
            l2=len(seq2)
            #se obtiene el numero de bases (nucleotidos) que contiene cada fragmento
            car_fr=contar_letras(seq1,l1)
            car_rv=contar_letras(seq2,l2)

            #se almacena en atributos
            attr_fr.append(car_fr)
            attr_rv.append(car_rv)
            
            #si el tamaño de ambos fragmentos no coincide, se rellena el menor con "N" 
            #(base ideterminada)
            max_len = max(len(seq1), len(seq2))
            if len(seq1) < max_len:
                seq1 += 'N' * (max_len - len(seq1))
            else:
                seq2 += 'N' * (max_len - len(seq2))
            
            #se almacenan ambas secuencias
            sequences_r1.append(seq1)
            sequences_r2.append(seq2)

            #se almacenan las calidades de los fragmentos
            #se rellena con 0 para las N añadidas anteriormente
            len_diff1 = len(seq1) - len(record1.letter_annotations["phred_quality"])
            if len_diff1 > 0:
                record1.letter_annotations["phred_quality"].extend([0] * len_diff1)
            len_diff2 = len(seq2) - len(record2.letter_annotations["phred_quality"])
            if len_diff2 > 0:
                record2.letter_annotations["phred_quality"].extend([0] * len_diff2)
            
            #adjunta las calidades por separado como listas
            qualities_f.append(record1.letter_annotations["phred_quality"])
            qualities_r.append(record2.letter_annotations["phred_quality"])
            
            #adjunta el id
            ids.append(R1_id_mod)
    
    #al final se retornan todos los datos juntos
    #se ve muy feo y es poco eficaz pero funciona
    return np.array(sequences_r1),np.array(sequences_r2),qualities_f,qualities_r,ids,attr_fr,attr_rv


La siguiente celda contiene una funcion que revisa si el los "ids" se encuentran presentes en el archivo FASTA 
"file1"

In [ ]:
def ver_si_contig_existe(file1,ids):
    existe = [[] for _ in range(len(ids))]  #se crea una lista existe del largo de ids
    id1=[]

    #se abre el archivo fasta y se obtienen solo los ids de todos los fragmentos
    for record3 in SeqIO.parse(file1, "fasta"):
        id1.append(record3.id.split()[0])
        
    #itera sobre cada ids[i] y compara con cada elemento en id1
    #si encuentra una coincidencia (ids[i] == id1[j]), establece existe[i] a 1 (posiblemente indicando verdadero)
    #rompe el bucle interno con break para pasar al siguiente ids[i]
    for i in range(0,len(ids)):
        for j in range(len(id1)):
            if(ids[i]==id1[j]):
                existe[i]=1
                break

    #recorre existe y convierte las listas vacías en 0
    for i in range(len(existe)):
        if isinstance(existe[i], list) and len(existe[i]) == 0:
            existe[i] = 0
    #retorna el resultado
    return existe

In [10]:
def encode_sequence_with_quality(sequence, quality_scores):
    mapping = {
            'A': [1, 0, 0, 0],
            'C': [0, 1, 0, 0],
            'G': [0, 0, 1, 0],
            'T': [0, 0, 0, 1],
            'M': [0.5, 0.5, 0, 0],
            'R': [0.5, 0, 0.5, 0],
            'W': [0.5, 0, 0, 0.5],
            'S': [0, 0.5, 0.5, 0],
            'Y': [0, 0.5, 0, 0.5],
            'K': [0, 0, 0.5, 0.5],
            'V': [0.33, 0.33, 0, 0.33],
            'H': [0.33, 0.33, 0.33, 0],
            'D': [0.33, 0, 0.33, 0.33],
            'B': [0, 0.33, 0.33, 0.33],
            'N': [0.25, 0.25, 0.25, 0.25],
        }

    encoded_sequence = []
    for i, base in enumerate(sequence):
        base_encoding =  mapping.get(base)
        if(quality_scores[i]):
            quality = quality_scores[i]

        else:
                quality=0
        normalized_quality = 1.0 if quality >= 50 else (quality - 30) / 20 if quality > 30 else 0.0

        encoded_sequence.append(base_encoding + [normalized_quality])
        
    return np.array(encoded_sequence)

def encode_fastq(sequences,quality,max_sequence_length=253):
    encoded_sequences = []
    for qual,seq in zip(quality,sequences):
        if len(seq) < max_sequence_length:
            seq += 'N' * (max_sequence_length - len(seq))
            qual.extend([0] * len(seq))# Pad sequence with 'N' to reach max length
        encoded_seq = encode_sequence_with_quality(seq[:max_sequence_length],qual)  # Truncate or pad to fixed length
        encoded_sequences.append(encoded_seq)
    return np.array(encoded_sequences)

In [11]:
def one_hot_encode_sequence(sequence):
    mapping = {
        'A': [1, 0, 0, 0],
        'C': [0, 1, 0, 0],
        'G': [0, 0, 1, 0],
        'T': [0, 0, 0, 1],
        'M': [0.5, 0.5, 0, 0],
        'R': [0.5, 0, 0.5, 0],
        'W': [0.5, 0, 0, 0.5],
        'S': [0, 0.5, 0.5, 0],
        'Y': [0, 0.5, 0, 0.5],
        'K': [0, 0, 0.5, 0.5],
        'V': [0.33, 0.33, 0, 0.33],
        'H': [0.33, 0.33, 0.33, 0],
        'D': [0.33, 0, 0.33, 0.33],
        'B': [0, 0.33, 0.33, 0.33],
        'N': [0.25, 0.25, 0.25, 0.25],
        '0': [0, 0, 0, 0]
    }

    return np.array(mapping[base] for base in sequence)

def encode_fastq_sequences(sequences):
    encoded_sequences = []
    for seq in sequences: # Pad sequence with 'N' to reach max length
        encoded_seq = one_hot_encode_sequence(seq)  # Truncate or pad to fixed length
        encoded_sequences.append(encoded_seq)
    return np.array(encoded_sequences)

In [12]:
def encode_sequence_with_quality(sequence, quality_scores):
    mapping = {
            'A': [1, 0, 0, 0],
            'C': [0, 1, 0, 0],
            'G': [0, 0, 1, 0],
            'T': [0, 0, 0, 1],
            'M': [0.5, 0.5, 0, 0],
            'R': [0.5, 0, 0.5, 0],
            'W': [0.5, 0, 0, 0.5],
            'S': [0, 0.5, 0.5, 0],
            'Y': [0, 0.5, 0, 0.5],
            'K': [0, 0, 0.5, 0.5],
            'V': [0.33, 0.33, 0, 0.33],
            'H': [0.33, 0.33, 0.33, 0],
            'D': [0.33, 0, 0.33, 0.33],
            'B': [0, 0.33, 0.33, 0.33],
            'N': [0.25, 0.25, 0.25, 0.25],
        }

    encoded_sequence = []
    for i, base in enumerate(sequence):
        base_encoding =  mapping.get(base)
        if(quality_scores[i]):
            quality = quality_scores[i]

        else:
                quality=0
        normalized_quality = 1.0 if quality >= 50 else (quality - 30) / 20 if quality > 30 else 0.0

        encoded_sequence.append(base_encoding + [normalized_quality])
        
    return np.array(encoded_sequence)

def encode_fastq(sequences,quality,max_sequence_length=253):
    encoded_sequences = []
    for qual,seq in zip(quality,sequences):
        if len(seq) < max_sequence_length:
            seq += 'N' * (max_sequence_length - len(seq))
            qual.extend([0] * len(seq))# Pad sequence with 'N' to reach max length
        encoded_seq = encode_sequence_with_quality(seq[:max_sequence_length],qual)  # Truncate or pad to fixed length
        encoded_sequences.append(encoded_seq)
    return np.array(encoded_sequences)

In [13]:

def promediar_cal(calidades):
    calidad=[]
    for cal in calidades:
        cal=np.array(cal)
        suma=np.sum(cal)
        calidad.append(round(suma/len(cal), 2))
    return calidad


    

In [15]:
def procesar_calidades2(calidades):
    qual1=0
    qual2=0
    for cal in calidades:
        
        if cal < 30:
            qual1=qual1+1
        else:
            qual2=qual2+1# Si el número es mayor que 50, devolver 1
    return qual1,qual2

def contar_cal(calidades):
    cal1=[]
    cal2=[]
    for cal in calidades:
        
        sum1,sum2=procesar_calidades2(cal)
        
        cal1.append(sum1)
        cal2.append(sum2)
    return cal1,cal2

In [21]:
batch_size = 10
a=[]
b=[]
for i in range(0, len(paths), batch_size):
    batch_paths = paths[i:i+batch_size]  # Get a batch of paths
    # Process each folder in the batch
    for j, path in enumerate(batch_paths):
        #combined=[]
        print(f"cargando archivos, it:{i+j+1}")
        paired_files = buscar_archivos_fastq(path)
        
        fasta_files= buscar_archivos_fasta(path)
        
        items_seqs= process_paired_fastq_files(paired_files[0], paired_files[1])

        #retorno
        #np.array(sequences_r1),    0
        # np.array(sequences_r2),   1
        #qualities_f,               2
        # qualities_r,              3
        # ids                       4
        # atributos fr              5
        # atributos rv              6
        cal_bu1,cal_ma1=contar_cal(items_seqs[2])
        cal_bu2,cal_ma2=contar_cal(items_seqs[3])
        #prom_cal1=promediar_cal(items_seqs[2])
        #prom_cal2=promediar_cal(items_seqs[3])
        """
        for item in items_seqs[2]:
            print(f"np prom_ {np.mean(np.array(item))}")
            print(f"np median_ {np.median(np.array(item))}")
            print(f"np desviacion_ {np.std(np.array(item))}")
            print(f"varianza {np.var(np.array(item))}")
            """
        atributos_fr=items_seqs[5]
        atributos_rv=items_seqs[6]
        
        
        for k in range(0,len(atributos_fr)):
            atributos_fr[k].append(cal_bu1[k])
            atributos_fr[k].append(cal_ma1[k])
            atributos_fr[k].append(np.mean(np.array(items_seqs[2][k])))
            atributos_fr[k].append(np.median(np.array(items_seqs[2][k])))
            atributos_fr[k].append(np.std(np.array(items_seqs[2][k])))
            atributos_fr[k].append(np.var(np.array(items_seqs[2][k])))
            
        for k in range(0,len(atributos_rv)):
            atributos_rv[k].append(cal_bu2[k])
            atributos_rv[k].append(cal_ma2[k])
            atributos_rv[k].append(np.mean(np.array(items_seqs[3][k])))
            atributos_rv[k].append(np.median(np.array(items_seqs[3][k])))
            atributos_rv[k].append(np.std(np.array(items_seqs[3][k])))
            atributos_rv[k].append(np.var(np.array(items_seqs[3][k])))
        #d=np.concatenate((atributos_fr,cal1),axis=0)
        #print(d[0])

        items_tgt=ver_si_contig_existe(fasta_files[0],items_seqs[4])
        
        y_train = np.ravel(items_tgt)
        
        x_train=np.concatenate((atributos_fr,atributos_rv),axis=1)
        
        #a.append(np.concatenate((atributos_fr,atributos_rv),axis=0))
        print(x_train.shape)

        # Aplanar X_seq2
        #a.append(X_combined_flat)
        #b.append(y_train)

        np.save(f'caracteristicas4/atributos_{i+j+1}.npy', np.array(x_train))
        np.save(f'caracteristicas4/resultados_{i+j+1}.npy', np.array(y_train))

        #if(i+j+1)==1:
#a = np.concatenate(a, axis=0)

#b = np.concatenate(b, axis=0)


cargando archivos, it:1
(19231, 24)
cargando archivos, it:2
(19231, 24)
cargando archivos, it:3
(7921, 24)
cargando archivos, it:4
(12222, 24)
cargando archivos, it:5
(13578, 24)
cargando archivos, it:6
(17790, 24)
cargando archivos, it:7
(7800, 24)
cargando archivos, it:8
(21082, 24)
cargando archivos, it:9
(7753, 24)
cargando archivos, it:10
(10325, 24)
cargando archivos, it:11
(4586, 24)
cargando archivos, it:12
(6152, 24)
cargando archivos, it:13
(12794, 24)
cargando archivos, it:14
(24237, 24)
cargando archivos, it:15
(9323, 24)
cargando archivos, it:16
(2927, 24)
cargando archivos, it:17
(9299, 24)
cargando archivos, it:18
(5892, 24)
cargando archivos, it:19
(7727, 24)
cargando archivos, it:20
(3714, 24)
cargando archivos, it:21
(3421, 24)
cargando archivos, it:22
(9081, 24)
cargando archivos, it:23
(13912, 24)
cargando archivos, it:24
(11063, 24)
cargando archivos, it:25
(5196, 24)
cargando archivos, it:26
(14629, 24)
cargando archivos, it:27
(3838, 24)
cargando archivos, it:28


possible_letters = ['A', 'C', 'G', 'T', 'M', 'R', 'W', 'S', 'Y', 'K', 'V', 'H', 'D', 'B', 'N']

In [16]:
print(x_train.shape)
#print(b.shape)

(6478, 18)


(54735, 2530)

(42312, 2530)

In [17]:
def train_model(model,input_1,input_2, targets, batch_size=32,epochs=2):

    return model.fit(
        x=[input_1, input_2],
        y=targets,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
        shuffle=True
    )

In [18]:
model = adapted_binary_model()

NameError: name 'adapted_binary_model' is not defined

In [ ]:
history = train_model(
        model,
        encoded_fr,
        encoded_rv,
        items_tgt,
        
        epochs=100
        #initial_epoch=initial_epoch
    )

Epoch 1/100


2023-12-23 23:06:36.561532: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_7/dropout_2/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


138/138 [==============================] - 3s 11ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/100
138/138 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/100
138/138 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/100
 64/138 [============>.................] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

KeyboardInterrupt: 

In [ ]:
X_seq1_flat = np.reshape(encoded_fr, (encoded_fr.shape[0], -1))  # Aplanar X_seq1
X_seq2_flat = np.reshape(encoded_rv, (encoded_rv.shape[0], -1))  # Aplanar X_seq2

# Concatenar los datos aplanados
X_combined_flat = np.concatenate((X_seq1_flat, X_seq2_flat), axis=1)



In [ ]:
y_train = np.ravel(items_tgt)  # Aplanar X_seq2
print(y_train.shape)

(5517,)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.2, random_state=42)

# Inicializar el clasificador SVM
svm = SVC(kernel='linear')

# Entrenar el modelo SVM
svm.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = svm.predict(X_test)

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Calcular precision, recall y f1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

# Reporte de clasificación
classification_rep = classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_rep)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

ValueError: Found input variables with inconsistent numbers of samples: [10947, 14764]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crear el heatmap de la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train1, X_test1, X_train2, X_test2, y_train, y_test = train_test_split(X_input1_flat, X_input2_flat, y_output_flat, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(a, b, test_size=0.2, random_state=42)

# Inicializar el regresor RandomForest
rf_regressor = RandomForestRegressor(n_estimators=30, random_state=42)

# Entrenar el modelo
rf_regressor.fit(X_train, y_train)
#rf_regressor.fit([X_train1, X_train2], y_train)

# Predecir en el conjunto de prueba
y_pred = rf_regressor.predict(X_test)
#predictions = rf_regressor.predict([X_test1, X_test2])


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.21148377739688326
R-squared: 0.02409219605763624


Mean Squared Error: 0.21148377739688326
R-squared: 0.02409219605763624

Mean Squared Error: 0.21793884920426165
R-squared: -0.01440627993256327


Mean Squared Error: 0.22119389956950966
R-squared: -0.029557059814445008

Mean Squared Error: 0.21692751518356335
R-squared: -0.009698979763494986

cambio de paradigma, dando conteo de letras y largo puro

Mean Squared Error: 0.17795166483968214
R-squared: 0.1788286525842

Mean Squared Error: 0.18141360431683015
R-squared: 0.1591024070537358

Mean Squared Error: 0.18141360431683015
R-squared: 0.1591024070537358


Mean Squared Error: 0.1755127574225682
R-squared: 0.17107820826153897


Mean Squared Error: 0.17888907529190715
R-squared: 0.15513233914734803

In [ ]:
print(y_pred.shape)
print(y_pred[0])

(1104,)
0.1625


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

batch_size = 5
  
for i in range(0, len(paths), batch_size):
    batch_paths = paths[i:i+batch_size]  # Get a batch of paths
    # Process each folder in the batch
    for j, path in enumerate(batch_paths):
        #combined=[]
        print("cargando archivos, it:",i+j+1)
        paired_files = buscar_archivos_fastq(path)
        fasta_files= buscar_archivos_fasta(path)
        
        items_seqs= process_paired_fastq_files(paired_files[0], paired_files[1])
        #retorno
        #np.array(sequences_r1),    0
        # np.array(sequences_r2),   1
        #qualities_f,               2
        # qualities_r,              3
        # ids                       4

        items_tgt=ver_si_contig_existe(fasta_files[0],fasta_files[1],items_seqs[4])
        
        encoded_fr=encode_fastq(items_seqs[0],items_seqs[2])
        encoded_rv=encode_fastq(items_seqs[1],items_seqs[3])
        
        encoded_fr = encoded_fr.reshape(-1, 253, 5, 1)
        encoded_rv= encoded_rv.reshape(-1, 253, 5, 1)
        items_tgt=np.array(items_tgt)
        items_tgt = items_tgt.reshape(-1, 1)
    
        #a=np.stack([encoded_fr,encoded_rv],axis=1)
        #print(encoded_tg.shape)
        #print(a.shape)
        print(encoded_fr.shape)
        print(encoded_rv.shape)
        print(items_tgt.shape)
        #print(cal_1.shape)
        #print(cal_1.shape)
        
        X_seq1_flat = np.reshape(encoded_fr, (encoded_fr.shape[0], -1))  # Aplanar X_seq1
        X_seq2_flat = np.reshape(encoded_rv, (encoded_rv.shape[0], -1))  # Aplanar X_seq2

        # Concatenar los datos aplanados
        X_combined_flat = np.concatenate((X_seq1_flat, X_seq2_flat), axis=1)
        
    

    


cargando archivos, it: 1
(5517, 253, 5, 1)
(5517, 253, 5, 1)
(5517, 1)
(5517,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 2
(12898, 253, 5, 1)
(12898, 253, 5, 1)
(12898, 1)
(12898,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 3
(18839, 253, 5, 1)
(18839, 253, 5, 1)
(18839, 1)
(18839,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 4
(71, 253, 5, 1)
(71, 253, 5, 1)
(71, 1)
(71,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 5
(4987, 253, 5, 1)
(4987, 253, 5, 1)
(4987, 1)
(4987,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 6
(5335, 253, 5, 1)
(5335, 253, 5, 1)
(5335, 1)
(5335,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 7
(7088, 253, 5, 1)
(7088, 253, 5, 1)
(7088, 1)
(7088,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 8
(11587, 253, 5, 1)
(11587, 253, 5, 1)
(11587, 1)
(11587,)
Mean Squared Error: 0.0
R-squared: 1.0
cargando archivos, it: 9
(5700, 253, 5, 1)
(5700, 25

In [ ]:
y_train = np.ravel(items_tgt)  # Aplanar X_seq2
print(y_train.shape)

X_train, X_test, y_train, y_test = train_test_split(X_combined_flat, y_train , test_size=0.2, random_state=42)

# Inicializar el regresor RandomForest

# Entrenar el modelo
rf_regressor.fit(X_train, y_train)
#rf_regressor.fit([X_train1, X_train2], y_train)

# Predecir en el conjunto de prueba
y_pred = rf_regressor.predict(X_test)
#predictions = rf_regressor.predict([X_test1, X_test2])

# Calcular el error (si es necesario)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


In [ ]:
for tree in rf_regressor.estimators_:  # Puedes cambiar el índice para visualizar otro árbol

# Grafica el árbol
    plt.figure(figsize=(20, 10))
    plot_tree(tree, filled=True, feature_names=[f'feature_{i}' for i in range(X_combined_flat.shape[1])], class_names=['0', '1'])
    plt.show()

In [ ]:
encoded_fr = np.load(f"seqs/encoded_fr_50.npy")
encoded_rv = np.load(f"seqs/encoded_rv_50.npy")
encoded_tg = np.load(f"seqs/encoded_tg_50.npy")


In [ ]:
items_seqs= process_paired_fastq_files(paired_files[0], paired_files[1])
#retorno
#np.array(sequences_r1),    0
# np.array(sequences_r2),   1
#qualities_f,               2
# qualities_r,              3
# ids                       4

items_tgt=ver_si_contig_existe(fasta_files[0],fasta_files[1],items_seqs[4])

encoded_fr=encode_fastq(items_seqs[0],items_seqs[2])
encoded_rv=encode_fastq(items_seqs[1],items_seqs[3])

encoded_fr = encoded_fr.reshape(-1, 253, 5, 1)
encoded_rv= encoded_rv.reshape(-1, 253, 5, 1)
items_tgt=np.array(items_tgt)
items_tgt = items_tgt.reshape(-1, 1)

#a=np.stack([encoded_fr,encoded_rv],axis=1)
#print(encoded_tg.shape)
#print(a.shape)
print(encoded_fr.shape)
print(encoded_rv.shape)
print(items_tgt.shape)
#print(cal_1.shape)
#print(cal_1.shape)

X_seq1_flat = np.reshape(encoded_fr, (encoded_fr.shape[0], -1))  # Aplanar X_seq1
X_seq2_flat = np.reshape(encoded_rv, (encoded_rv.shape[0], -1))  # Aplanar X_seq2

# Concatenar los datos aplanados
X_combined_flat = np.concatenate((X_seq1_flat, X_seq2_flat), axis=1)

y_train = np.ravel(items_tgt)  # Aplanar X_seq2
print(y_train.shape)

(4987, 253, 5, 1)
(4987, 253, 5, 1)
(4987, 1)
(4987,)


In [ ]:
print(f"cargando archivos, it:{i+j+1}")
paired_files = buscar_archivos_fastq("../tutorialM/datos/sets/Folder_50")

fasta_files= buscar_archivos_fasta("../tutorialM/datos/sets/Folder_50")

items_seqs= process_paired_fastq_files(paired_files[0], paired_files[1])

#retorno
#np.array(sequences_r1),    0
# np.array(sequences_r2),   1
#qualities_f,               2
# qualities_r,              3
# ids                       4
# atributos fr              5
# atributos rv              6

atributos_fr=items_seqs[5]
atributos_rv=items_seqs[6]


        
items_tgt=ver_si_contig_existe(fasta_files[0],fasta_files[1],items_seqs[4])

y_prueba = np.ravel(items_tgt)
x_prueba=(np.concatenate((atributos_fr,atributos_rv),axis=1))

# Aplanar X_seq2
#a.append(X_combined_flat)
#b.append(y_train)

cargando archivos, it:1


In [ ]:
print(x_prueba.shape)
print(y_prueba.shape)

(14764, 32)
(14764,)


In [ ]:
y_pred = rf_regressor.predict(x_prueba)
print(y_pred.shape)

(14764,)


In [ ]:
print(y_prueba.shape)

(14764,)


In [ ]:
mse = mean_squared_error(y_prueba, y_pred)
r2 = r2_score(y_prueba, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 0.22436336008358926
R-squared: 0.014542916798066075


In [ ]:
print(y_pred.shape)
print(y_prueba.shape)

(14764,)
(14764,)


In [ ]:
count=0
count1=0
count2=0
for i,j in zip(y_pred,y_prueba):
    if i>0.5:
        aux=1
    else:
        aux=0
    if aux==j:
        count=count+1
    if aux==1:
        count1=count1+1
    if j==1:
        count2=count2+1
print(f'similitudes {count}')
print(f'elementos pre {count1}')
print(f'elementos og {count2}')

print(f"totales: {len(y_prueba)}")
print(f"delta el {count2-count1}")

similitudes 9999
elementos pre 3027
elementos og 5176
totales: 14764
delta el 2149


similitudes 9999
elementos pre 3027
elementos og 5176
totales: 14764
delta el 2149

similitudes 9417
elementos pre 1493
elementos og 5176
totales: 14764
delta el 3683

cambio de paragdigma

similitudes 4718
elementos pre 1459
elementos og 1620
4987

prueba 3 archivos

17929
5342
5960
18839
 unos
 

17929
13497
12879
18839
ceros


1345
1046
12879
18839
ceros


1345
301
5960
18839
unos